In [1]:
import sys  
import os
import pandas as pd
import io
sys.path.insert(0, '../../../../scripts/')
sys.path.insert(0, '../../../../dataset/')
import embedding_reader
import custom_keras_metrics
import tensorflow as tf
import keras.backend as K
import os
import time
import gc
import glob
import wget
import zipfile
import numpy as np
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import LSTM
from tensorflow.keras.preprocessing import text, sequence
from keras.layers import Reshape, Flatten, Concatenate, Dropout, SpatialDropout1D
from tensorflow.keras.optimizers import Adam
from keras.models import Model
from keras import backend as K
from tensorflow.keras.layers import Layer
from keras import initializers, regularizers, constraints, optimizers, layers
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from keras.layers import *
from keras.models import *
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.initializers import *
from keras.optimizers import *
from keras.callbacks import *
from keras import metrics
from tqdm import tqdm

2023-10-23 00:10:08.579126: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-10-23 00:10:08.579161: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
#wget.download("http://143.107.183.175:22980/download.php?file=embeddings/glove/glove_s300.zip", out="../../../../embeddings/")

In [3]:
#with zipfile.ZipFile("../../../../embeddings/glove_s300.zip", 'r') as zip_ref:
    #zip_ref.extractall("../../../../embeddings/")

In [2]:
MAX_LENGTH = 300
dataframe = pd.read_csv('../../../../dataset/HateBR.csv')
y = dataframe['offensive_language']

In [3]:
x = dataframe['instagram_comments']

In [4]:
token = Tokenizer()
token.fit_on_texts(x)
sequences = token.texts_to_sequences(x)
x = pad_sequences(sequences,maxlen=MAX_LENGTH)
vocabulary_size = len(token.word_index)+1

In [5]:
embedding_vector = embedding_reader.read_glove_file()
embedding_matrix = np.zeros((vocabulary_size,MAX_LENGTH))
for word,i in tqdm(token.word_index.items()):
    embedding_value = embedding_vector.get(word)
    if embedding_value is not None:
        embedding_matrix[i] = embedding_value

100%|██████████| 11530/11530 [00:00<00:00, 288135.21it/s]


In [6]:
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.8, test_size=0.2)

In [7]:
lstm_model = Sequential()
lstm_model.add(Embedding(vocabulary_size,MAX_LENGTH,weights=[embedding_matrix],input_length=x.shape[1],trainable=False))
lstm_model.add(Dropout(0.25))
lstm_model.add(LSTM(64))
lstm_model.add(Dropout(0.5))
lstm_model.add(Dense(1))
lstm_model.add(Activation('sigmoid'))
lstm_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', 'binary_accuracy', custom_keras_metrics.f1_score])

2023-10-23 00:11:43.814497: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-10-23 00:11:43.814541: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-10-23 00:11:43.814572: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (codespaces-fbf4a0): /proc/driver/nvidia/version does not exist
2023-10-23 00:11:43.814896: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [8]:
model = lstm_model.fit(x_train, y_train, epochs=20, batch_size=128, validation_data=(x_test, y_test))

Epoch 1/20


44/44 [==============================] - 23s 490ms/step - loss: 0.5663 - accuracy: 0.7109 - binary_accuracy: 0.7109 - f1_score: 0.7079 - val_loss: 0.4877 - val_accuracy: 0.7850 - val_binary_accuracy: 0.7850 - val_f1_score: 0.8019
Epoch 2/20
44/44 [==============================] - 21s 482ms/step - loss: 0.4369 - accuracy: 0.8075 - binary_accuracy: 0.8075 - f1_score: 0.8030 - val_loss: 0.3890 - val_accuracy: 0.8450 - val_binary_accuracy: 0.8450 - val_f1_score: 0.8314
Epoch 3/20
44/44 [==============================] - 21s 483ms/step - loss: 0.3901 - accuracy: 0.8313 - binary_accuracy: 0.8313 - f1_score: 0.8273 - val_loss: 0.3586 - val_accuracy: 0.8571 - val_binary_accuracy: 0.8571 - val_f1_score: 0.8525
Epoch 4/20
44/44 [==============================] - 21s 478ms/step - loss: 0.3555 - accuracy: 0.8389 - binary_accuracy: 0.8389 - f1_score: 0.8344 - val_loss: 0.3458 - val_accuracy: 0.8593 - val_binary_accuracy: 0.8593 - val_f1_score: 0.8520
Epoch 5/20
44/44 [=============================

In [9]:
results = lstm_model.evaluate(x_test, y_test, batch_size=128)

for name, value in zip(lstm_model.metrics_names, results):
  print(name, ': ', value)
print()

 1/11 [=>............................] - ETA: 1s - loss: 0.2398 - accuracy: 0.8906 - binary_accuracy: 0.8906 - f1_score: 0.9000

11/11 [==============================] - 2s 155ms/step - loss: 0.3437 - accuracy: 0.8671 - binary_accuracy: 0.8671 - f1_score: 0.8677
loss :  0.34367650747299194
accuracy :  0.8671428561210632
binary_accuracy :  0.8671428561210632
f1_score :  0.8676944375038147



In [12]:
#import joblib

#model_filename = '../../../models/generated-models-base-paper/offensive_lstm_embeddings_model.pkl'
#joblib.dump(model, model_filename)

: 